In [1]:
import pandas as pd
from datetime import timedelta
import pandas as pd
import numpy as np
from rpy2.robjects import pandas2ri
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from scipy.stats import f
from sklearn import metrics
from sklearn.model_selection import KFold
import rpy2
import numpy as np

import os
os.chdir(os.getcwd()+"/../..")
main_folder=os.getcwd()

pandas2ri.activate()
mgcv = importr('mgcv')
VIF=importr('VIF')

In [2]:
statsf = importr('stats',robject_translations = {'format_perc': 'format_dot_perc'})
risknames = [ 'aki_all','MV_duration','cardiovasc_comp_comb', 'ICU_duration', 'neuro_delirium_comb', 'post_op_mech_proc_comb', 'sepsis_comb', 'venos_throm_comb']

In [20]:
df=pd.read_csv(main_folder+"/Supplement data/mortality_12_12_17.csv")
for riskname in risknames:
    print (" mortlaility for", riskname)
    #os.chdir(main_folder+"/Models/"+riskname+" .Rdata")
    ro.globalenv['riskname']=riskname
    ro.globalenv['main_folder']=main_folder
    ro.r['load'](main_folder+"/Models/"+riskname+" .Rdata")
    best_model_cvcomp=ro.globalenv['best_model']
    #os.chdir(main_folder)
    #ro.r('load(best_model,file=paste(location,".Rdata"))')
    #testdata=pd.read_csv(main_folder+"/Val_Cohort/test_cohort_"+riskname+".csv")
    devdata=pd.read_csv(main_folder+"/Dev_Cohort/development_cohort_"+riskname+".csv")
    ro.globalenv['r_df_cvcomp'] = pandas2ri.py2ri(devdata)
    r_df_cvcomp=ro.r('r_df_cvcomp')
    pred_score_cvcomp=statsf.predict(best_model_cvcomp,r_df_cvcomp, type="response")
    devdata.rename(columns={'outcome':riskname}, inplace=True)
    devdata[riskname]=pred_score_cvcomp
    devdata=devdata[["encounter_deiden_id",riskname]]
    df=pd.merge(df,devdata,on="encounter_deiden_id",how="outer")
df_train=df[pd.notnull(df["MV_duration"])]
    

(' mortlaility for', 'aki_all')
(' mortlaility for', 'MV_duration')
(' mortlaility for', 'cardiovasc_comp_comb')
(' mortlaility for', 'ICU_duration')
(' mortlaility for', 'neuro_delirium_comb')
(' mortlaility for', 'post_op_mech_proc_comb')
(' mortlaility for', 'sepsis_comb')
(' mortlaility for', 'venos_throm_comb')


In [21]:
df=pd.read_csv(main_folder+"/Supplement data/mortality_12_12_17.csv")
for riskname in risknames:
    print (" mortlaility for", riskname)
    #os.chdir(main_folder+"/Models/"+riskname+" .Rdata")
    ro.globalenv['riskname']=riskname
    ro.globalenv['main_folder']=main_folder
    ro.r['load'](main_folder+"/Models/"+riskname+" .Rdata")
    best_model_cvcomp=ro.globalenv['best_model']
    #os.chdir(main_folder)
    #ro.r('load(best_model,file=paste(location,".Rdata"))')
    #testdata=pd.read_csv(main_folder+"/Val_Cohort/test_cohort_"+riskname+".csv")
    devdata=pd.read_csv(main_folder+"/Val_Cohort/test_cohort_"+riskname+".csv")
    ro.globalenv['r_df_cvcomp'] = pandas2ri.py2ri(devdata)
    r_df_cvcomp=ro.r('r_df_cvcomp')
    pred_score_cvcomp=statsf.predict(best_model_cvcomp,r_df_cvcomp, type="response")
    devdata.rename(columns={'outcome':riskname}, inplace=True)
    devdata[riskname]=pred_score_cvcomp
    devdata=devdata[["encounter_deiden_id",riskname]]
    df=pd.merge(df,devdata,on="encounter_deiden_id",how="outer")
df_val=df[pd.notnull(df["MV_duration"])]
    

(' mortlaility for', 'aki_all')
(' mortlaility for', 'MV_duration')
(' mortlaility for', 'cardiovasc_comp_comb')
(' mortlaility for', 'ICU_duration')
(' mortlaility for', 'neuro_delirium_comb')
(' mortlaility for', 'post_op_mech_proc_comb')
(' mortlaility for', 'sepsis_comb')
(' mortlaility for', 'venos_throm_comb')


In [25]:
df_val.shape
df_train.shape


(13572, 15)

In [51]:
df_train_ESRD=df_train[pd.isnull(df_train["aki_all"])]
df_train_NotESRD=df_train[pd.notnull(df_train["aki_all"])]
df_val_ESRD=df_val[pd.isnull(df_val["aki_all"])]
df_val_NotESRD=df_val[pd.notnull(df_val["aki_all"])]

In [27]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection  import GridSearchCV 
from sklearn.externals import joblib

In [90]:
def run_rf_model(data, colnames): 
    Kfold_value=5 #number of folds
    data=data[pd.notnull(data).all(axis=1)]
    features =data[data.columns.difference([colnames])]
    print("Running Random_Forest_Classifire")
    clf=RFC(n_jobs=2)
    param_grid={"n_estimators"      : [100, 300, 500], #tuning the random forest with these parameters
                "criterion"         : ["gini", "entropy"],
                'max_features': [ 'sqrt', None, 5, 6, 7],
                "max_depth"         : [None ,10, 20]}
    grid_search = GridSearchCV(clf, param_grid, scoring='roc_auc',n_jobs=2, cv=Kfold_value) #acu metric used to find the best model
    grid_search.fit(features, data[colnames])
    model=grid_search.best_estimator_
    acu=grid_search.best_score_
    return model,acu

In [94]:
models=["mort_status_30d","mort_status_90d","mort_status_6m","mort_status_1y","mort_status_2y"]
feature_names= ['cardiovasc_comp_comb','ICU_duration','neuro_delirium_comb','post_op_mech_proc_comb','sepsis_comb',
                'venos_throm_comb','MV_duration']

for model in models:
    mdl,acu=run_rf_model(df_train_ESRD[feature_names+[model]],model)
    print("acu for "+model+' is: '+str(acu))
    joblib.dump(mdl, main_folder+'/M_Models/'+model+'_ESRD_RandomForestClassifier.pkl')

Running Random_Forest_Classifire
acu for mort_status_30d is: 0.847397937137
Running Random_Forest_Classifire
acu for mort_status_90d is: 0.750654504957
Running Random_Forest_Classifire
acu for mort_status_6m is: 0.721169037952
Running Random_Forest_Classifire
acu for mort_status_1y is: 0.712248020801
Running Random_Forest_Classifire
acu for mort_status_2y is: 0.642972987375


In [93]:
models=["mort_status_30d","mort_status_90d","mort_status_6m","mort_status_1y","mort_status_2y"]
feature_names= ['cardiovasc_comp_comb','ICU_duration','neuro_delirium_comb','post_op_mech_proc_comb','sepsis_comb',
                'venos_throm_comb','aki_all','MV_duration']

for model in models:
    mdl,acu=run_rf_model(df_train_NotESRD[feature_names+[model]],model)
    print("acu for "+model+' is: '+str(acu))
    joblib.dump(mdl, main_folder+'/M_Models/'+model+'_NotESRD_RandomForestClassifier.pkl')

Running Random_Forest_Classifire
acu for mort_status_30d is: 0.842831295936
Running Random_Forest_Classifire
acu for mort_status_90d is: 0.835359418125
Running Random_Forest_Classifire
acu for mort_status_6m is: 0.827287000159
Running Random_Forest_Classifire
acu for mort_status_1y is: 0.810304522809
Running Random_Forest_Classifire
acu for mort_status_2y is: 0.788904446298


In [29]:
def find_auc(model,data):
    ro.globalenv['model']=model
    ro.globalenv['test']=data
    e=ro.r("predict(model,newdata=test,type=\"response\")")
    
    fpr, tpr, thresholds = metrics.roc_curve(data['outcome'], e)
    return metrics.auc(fpr, tpr)

In [30]:
os.chdir(main_folder+"/M_Models")

In [41]:
l=['cardiovasc_comp_comb','ICU_duration','neuro_delirium_comb','post_op_mech_proc_comb','sepsis_comb',
                'venos_throm_comb','aki_all','MV_duration']

In [59]:
ll=['mort_status_30d',
 'mort_status_90d',
 'mort_status_6m',
 'mort_status_1y',
 'mort_status_2y']

In [55]:
df_val_NotESRD[l].isnull().any()

cardiovasc_comp_comb      False
ICU_duration              False
neuro_delirium_comb       False
post_op_mech_proc_comb    False
sepsis_comb               False
venos_throm_comb          False
aki_all                   False
MV_duration               False
dtype: bool

In [86]:
for x,outcome in zip(mort_status_NotESRD,ll):
    #for row in df_val_NotESRD.rows:
    e=x.predict(df_val_NotESRD[l])
    fpr, tpr, thresholds = metrics.roc_curve(df_val_NotESRD[outcome], e)
    print metrics.auc(fpr, tpr)
    #print(df_val_NotESRD[outcome].shape)
    #print e

0.5
0.499812453113
0.501506024096
0.50295398565
0.512333563006


In [84]:
np.mean(e)

0.0062769010043041609

In [79]:
print e[0]

[ 0.92333333  0.07666667]


In [35]:
for x,outcome in zip(list(df_train),list(df_train)):
    print x,outcome

encounter_deiden_id encounter_deiden_id
admission_to_death_time admission_to_death_time
mort_status_30d mort_status_30d
mort_status_90d mort_status_90d
mort_status_6m mort_status_6m
mort_status_1y mort_status_1y
mort_status_2y mort_status_2y
aki_all aki_all
MV_duration MV_duration
cardiovasc_comp_comb cardiovasc_comp_comb
ICU_duration ICU_duration
neuro_delirium_comb neuro_delirium_comb
post_op_mech_proc_comb post_op_mech_proc_comb
sepsis_comb sepsis_comb
venos_throm_comb venos_throm_comb


In [ ]:
for y in mort_status_ESRD:
    print ("Gam for", riskname)
    print "AUC for "+y+": ",find_auc(y,df_val)

In [ ]:
def find_auc(model,data):
    e=model.predict_proba(data)
    fpr, tpr, thresholds = metrics.roc_curve(data['outcome'], e)
    print metrics.auc(fpr, tpr)
